# Hot-Spot detection

Starting equations of note;

\begin{equation}
    R_{A,thermal} = \tau_{A}[f_{h}L(\lambda_{A},T_{h}) + (1-f_{h})L(\lambda_{A},T_{c})] \\
    R_{B,thermal} = \tau_{B}[f_{h}L(\lambda_{B},T_{h}) + (1-f_{h})L(\lambda_{B},T_{c})] \\
    R_{C,thermal} = \tau_{C}[f_{h}L(\lambda_{C},T_{h}) + (1-f_{h})L(\lambda_{B},T_{c})]
\end{equation}

Equation 1:
\begin{equation}
    L_{\lambda, T} = \frac{\epsilon \times C_{1} \times \lambda ^{-5}}{\pi \times (e^{\frac{C_{2}}{\lambda \times T_{i}}} - 1)}
\end{equation}


Equation 2: Surface fraction
\begin{equation}
    f_{h} = \frac{K - L(\lambda, T_{c})}{L(\lambda, T_{melt}) - L(\lambda, T_{c min})} \\
    K = \frac{R}{\tau}
\end{equation}


Equation 3: Integrated Temperature
\begin{equation}
    T_{i} = \frac{C_{2}}{\lambda \times ln(\frac{\tau \times \epsilon \times C_{1}}{\pi \times \lambda^{5} \times R} + 1)}
\end{equation}

Equation 4: Effective Temperature
\begin{equation}
    T_{e} = [f_{h} \times T_{h}^{4} + (1 - f_{h})T_{c}^{4}]^{\frac{1}{4}}
\end{equation}

And what everything means
Whats what in the equations above...
* $\tau$ = atmospheric spectral transmission coefficient
* $f_{h}$ = surface fraction
* $\epsilon$ = emissivity of the radiative surface
* $C_{x}$ are constants
* T = temperature
* R = thermal radiance

Define global contants

In [11]:
emissivity = 0.80
transmissivity = 0.64
c_1 = 374200000.0
c_2 = 14388.0
t_melt = 1323.0

# arbitrary small number
epsilon = 1e-6

## Equations as functions

In [2]:
import numpy as np

def L(lam, temp):
    """
    Equation 1 above
    @param lam wavelength
    @param temp temperature in K
    """
    numerator = emissivity * c_1 * lam **-5
    denominator = np.pi * (np.exp(c_2 / (lam * temp)) - 1)
    return numerator / denominator

def R(lam, temp):
    """Not actually used
    Equation 1 times transmissivity
    @param lam wavelength
    @param temp temperature in K
    """
    return transmissivity * L(lam, temp)

def calculate_fh(K, L_lam_T_c, L_lam_T_melt):
    """
    Equation 2 above
    @param K therm / transmissivity
    @param L_lam_T_c L(lam, T_c_min)
    @param L_lam_5_T_melt L(lam_5, T_melt)
    """
    numerator = K - L_lam_T_c
    denominator = L_lam_T_melt - L_lam_T_c
    return numerator / denominator

def calculateTi(r_therm, lam):
    """Caculate integrated temperature
    Equation 3 above.
    @param R_therm therm
    @param lam wavelength
    """
    log_num = transmissivity * emissivity * c_1
    log_den = np.pi * lam**5 * r_therm
    if log_num/log_den < 0 :
        return None
    return c_2 / (lam * np.log(np.fabs(log_num/log_den) + 1))

def calculateTe(f_h, t_h, t_c):
    """Calculate the effective temperature
    Equation 4 above.
    @param f_h surface fraction
    @param t_h magmatic temperature
    @param t_c average temperature
    """
    a = f_h * t_h ** 4 
    b = (1 - f_h) * t_c ** 4
    return (a + b) ** 0.25

def calculateQ(A, e, sigma, t):
    """Calculate total radiant flux
    """
    return A * e * sigma * t ** 4

def same_sign(x, y):
    """
    @param x 
    @param y 
    @return True is x and y are of the same sign, else False
    """
    return (x > 0 and y > 0) or (x < 0 and y < 0)

def find_tc(t_c_min, t_c_max,
           l_tmelt_A, l_tmelt_B,
           lambda_A, lambda_B,
           k_A, k_B,
           fh_min_A, fh_max_A,
           fh_min_B, fh_max_B):
    """Recursive approach to finding tc    
    """
    if abs(t_c_min - t_c_max) > epsilon:
        t_c_mid = (t_c_max + t_c_min) / 2.0 # Calculate mid point
        fh_A_mid = calculate_fh(k_A, L(lambda_A, t_c_mid), l_tmelt_A)
        fh_B_mid = calculate_fh(k_B, L(lambda_B, t_c_mid), l_tmelt_B)
            
        # If the both fh are the same sign
        if same_sign(fh_A_mid - fh_B_mid, fh_min_A - fh_min_B):
            # True if...
            # Case 1: Both negative; fh_B_mid > fh_A_mid and fh_B_min > fh_A_min
            # Case 2: Both positive; fh_A_mid > fh_B_mid and fh_A_min > fh_B_min
            # Set low point to mid point
            fh_min_A = fh_A_mid
            fh_min_B = fh_B_mid
            t_c_min = t_c_mid
        else:
            # If False
            # Set high point to be mid point
            fh_max_A = fh_A_mid
            fh_max_B = fh_B_mid
            t_c_max = t_c_mid
            
        # Here is the recursion
        t_c_min, t_c_max, fh_min_A, fh_max_A, fh_min_B, fh_max_B = find_tc(t_c_min, t_c_max,
                                                                           l_tmelt_A, l_tmelt_B,
                                                                           lambda_A, lambda_B,
                                                                           k_A, k_B,
                                                                           fh_min_A, fh_max_A,
                                                                           fh_min_B, fh_max_B)
        
    return t_c_min, t_c_max, fh_min_A, fh_max_A, fh_min_B, fh_max_B

def calculateTh(lambda_A, tau_A, t_c, r_A, f_h):
    """Rearrange starting equations to calculate t_h
    @param lambda_A wavelength
    @param tau_A
    @param t_c
    @param r_A
    @param f_h
    """
    l_a = L(lambda_A, t_c)
    l_th = ((r_A / tau_A) - (1 - f_h) * l_a) / f_h
    t_h =  (c_2 / np.pi) * np.log((emissivity * c_1 * lambda_A ** -5) / (np.pi * l_th ) + 1)
    return t_h

## Handle Output CSV

In [3]:
import csv # slightly easier to use than xlrd

def create_csv(outname):
    """Fill a row with titles
    NOTE will overwrite any existing csv with the same name
    Uses global variable row_titles
    @param outname csv output name
    """
    with open(outname, 'w', newline='') as csvfile:
        thewriter = csv.writer(csvfile, delimiter=',')
        thewriter.writerow(row_titles)

def insert_row(outname, rowinfo):
    """Just insert info on the next line
    @param outname csv output name
    @param rowinfo list of information to fill that row with
    """
    with open(outname, 'a', newline='') as csvfile:
        thewriter = csv.writer(csvfile, delimiter=',')
        thewriter.writerow(rowinfo)    

## Dual Band

In [13]:
def pixelDualBand(rad_A, rad_B):
    """Original code
    Slightly rewritten to make easier to read
    Loop over arrays and fill a row
    Made generic by A and B
    @param rawA array of radient pixel values
    @param rawB array of radient pixel values
    """

    for i in range(rad_A.size):
        
        # Have to reset these after each loop
        t_c_min =  356.0 
        t_c_max = 650.0
        
        # raw_A
        rad_pixel_A = rad_A[i]
        rad_thermal_A = rad_pixel_A - bg_A # subtract background to leave thermal radience
        ti_A = calculateTi(rad_thermal_A, lambda_A) # calculate integrated temperature
        r_A_saturated = rad_thermal_A > r_A_max # Determine if saturated
        r_A_too_small = rad_thermal_A < r_A_min # Determine if too small
        l_tmelt_A = L(lambda_A, t_melt) # Do once as used multiple times
        k_A = rad_thermal_A / transmissivity # Do once as used multiple times
        
        # raw_B
        rad_pixel_B = rad_B[i]        
        rad_thermal_B = rad_pixel_B - bg_B # subtract background to leave thermal radience
        ti_B = calculateTi(rad_thermal_B, lambda_B) # calculate integrated temperature
        r_B_saturated = rad_thermal_B > r_B_max # Determine if saturated
        r_B_too_small = rad_thermal_B < r_B_min # Determine if too small
        l_tmelt_B = L(lambda_B, t_melt) # Do once as used multiple times
        k_B = rad_thermal_B / transmissivity # Do once as used multiple times        
        
        # Starting points
        fh_min_A = calculate_fh(k_A, L(lambda_A, t_c_min), l_tmelt_A)
        fh_max_A = calculate_fh(k_A, L(lambda_A, t_c_max), l_tmelt_A)
        
        fh_min_B = calculate_fh(k_B, L(lambda_B, t_c_min), l_tmelt_B)
        fh_max_B = calculate_fh(k_B, L(lambda_B, t_c_max), l_tmelt_B)

        # Find mid point
        # Definately could be replaced by a scipy function
        # Halve search region each time
        t_c_min, t_c_max, fh_min_A, fh_max_A, fh_min_B, fh_max_B = find_tc(t_c_min, t_c_max,
                                                                           l_tmelt_A, l_tmelt_B,
                                                                           lambda_A, lambda_B,
                                                                           k_A, k_B,
                                                                           fh_min_A, fh_max_A,
                                                                           fh_min_B, fh_max_B)
        
        # Once difference < epsilon    
        # t_c = midpoint
        t_c = (t_c_min + t_c_max) / 2.0
        t_c_min, t_c_max = 356.0, 650.0 # Reset variables
        
        # Determine if there is convergence
        if min(abs(t_c - t_c_min), abs(t_c - t_c_max)) < 2*epsilon:
            # If No Convergence
            # ie t_c - t_c_min/max < 2*epsilon
            # Dual band solution fails
            f_h = 0.0
            
            if not r_A_saturated and not r_B_saturated and not r_B_too_small:
                # If both bands are unsaturated and rad B is not too small
                # t_central is integrated T of band B
                t_c = ti_B
            else:
                # Else set t_c to integrated T of band A
                t_c = ti_A
                
        else:
            # If Convergence
            # Calculate fraction
            fh_A = calculate_fh(k_A, L(lambda_A, t_c), l_tmelt_A)
            fh_B = calculate_fh(k_B, L(lambda_B, t_c), l_tmelt_B)
            f_h = (fh_A + fh_B) / 2.0
        
        # Calculate effective temperature
        if f_h > 0.0: # Solution found
            t_h = t_melt
            t_e = calculateTe(f_h, t_h, t_c)       
        else:
            t_e = t_c
            
        # Calculate Q
        q = calculateQ(A, e_q, sigma, t_e)
            
        # Create Row
        rowinfo = []
        rowinfo.append(rad_pixel_A)
        rowinfo.append(rad_pixel_B)
        rowinfo.append(rad_thermal_A)
        rowinfo.append(rad_thermal_B)
        rowinfo.append(ti_A)
        rowinfo.append(ti_B)
        rowinfo.append('yes' if r_A_saturated else 'no')
        rowinfo.append('yes' if r_B_saturated else 'no')
        rowinfo.append('yes' if r_A_too_small else 'no')
        rowinfo.append('yes' if r_B_too_small else 'no')
        rowinfo.append(f_h)
        rowinfo.append(1.0 - f_h)
        rowinfo.append(t_c)
        rowinfo.append(t_e)
        rowinfo.append(q)
    
        insert_row(outfile, rowinfo)

## TriBand

In [5]:
def pixelTriBand(rad_A, rad_B, rad_C):
    """
    Three band approach
    @param raw_A array of radient pixel values
    @param raw_B array of radient pixel values
    @param raw_C array of radient pixel values
    """
    
    for i in range(rad_A.size):
        
        # Have to reset these after each loop
        t_c_min =  356.0 
        t_c_max = 650.0
        
        # raw_A
        rad_pixel_A = rad_A[i]
        rad_thermal_A = rad_pixel_A - bg_A # subtract background to leave thermal radience
        ti_A = calculateTi(rad_thermal_A, lambda_A) # calculate integrated temperature
        r_A_saturated = rad_thermal_A > r_A_max # Determine if saturated
        r_A_too_small = rad_thermal_A < r_A_min # Determine if too small
        l_tmelt_A = L(lambda_A, t_melt) # Do once as used multiple times
        k_A = rad_thermal_A / transmissivity # Do once as used multiple times
        
        # raw_B
        rad_pixel_B = rad_B[i]        
        rad_thermal_B = rad_pixel_B - bg_B # subtract background to leave thermal radience
        ti_B = calculateTi(rad_thermal_B, lambda_B) # calculate integrated temperature
        r_B_saturated = rad_thermal_B > r_B_max # Determine if saturated
        r_B_too_small = rad_thermal_B < r_B_min # Determine if too small
        l_tmelt_B = L(lambda_B, t_melt) # Do once as used multiple times
        k_B = rad_thermal_B / transmissivity # Do once as used multiple times    
        
        # raw_C
        rad_pixel_C = rad_C[i]        
        rad_thermal_C = rad_pixel_C - bg_C # subtract background to leave thermal radience
        ti_C = calculateTi(rad_thermal_B, lambda_C) # calculate integrated temperature
        r_C_saturated = rad_thermal_C > r_C_max # Determine if saturated
        r_C_too_small = rad_thermal_C < r_C_min # Determine if too small
        l_tmelt_C = L(lambda_C, t_melt) # Do once as used multiple times
        k_C = rad_thermal_C / transmissivity # Do once as used multiple times  
        
        # Starting points
        fh_min_A = calculate_fh(k_A, L(lambda_A, t_c_min), l_tmelt_A)
        fh_max_A = calculate_fh(k_A, L(lambda_A, t_c_max), l_tmelt_A)
        
        fh_min_B = calculate_fh(k_B, L(lambda_B, t_c_min), l_tmelt_B)
        fh_max_B = calculate_fh(k_B, L(lambda_B, t_c_max), l_tmelt_B)
        
        fh_min_C = calculate_fh(k_C, L(lambda_C, t_c_min), l_tmelt_C)
        fh_max_C = calculate_fh(k_C, L(lambda_C, t_c_max), l_tmelt_C)

        # Find t_c
        # Logic for this
        # Order to try;
        # 1. AB 
        # 2. AC
        # 3. BC
        
        # AB
        t_c_min, t_c_max, fh_min_A, fh_max_A, fh_min_B, fh_max_B = find_tc(t_c_min, t_c_max,
                                                                           l_tmelt_A, l_tmelt_B,
                                                                           lambda_A, lambda_B,
                                                                           k_A, k_B,
                                                                           fh_min_A, fh_max_A,
                                                                           fh_min_B, fh_max_B)
        
        t_c = (t_c_min + t_c_max) / 2.0
        t_c_min, t_c_max = 356.0, 650.0 # Reset variables
        
        # Determine if there is convergence in AB
        if min(abs(t_c - t_c_min), abs(t_c - t_c_max)) < 2*epsilon:
            # If no AB convergence 
            # Try AC
            fh_min_A = calculate_fh(k_A, L(lambda_A, t_c_min), l_tmelt_A) # Reset fh_min_A
            fh_max_A = calculate_fh(k_A, L(lambda_A, t_c_max), l_tmelt_A) # Reset fh_max_A
            
            t_c_min, t_c_max, fh_min_A, fh_max_A, fh_min_C, fh_max_C = find_tc(t_c_min, t_c_max,
                                                                               l_tmelt_A, l_tmelt_C,
                                                                               lambda_A, lambda_C,
                                                                               k_A, k_C,
                                                                               fh_min_A, fh_max_A,
                                                                               fh_min_C, fh_max_C)
            
            if min(abs(t_c - t_c_min), abs(t_c - t_c_max)) < 2*epsilon:
                # If no AC convergence
                # Try BC
                fh_min_B = calculate_fh(k_B, L(lambda_B, t_c_min), l_tmelt_B) # Reset fh_min_B
                fh_max_B = calculate_fh(k_B, L(lambda_B, t_c_max), l_tmelt_B) # Reset fh_max_B
                fh_min_C = calculate_fh(k_C, L(lambda_C, t_c_min), l_tmelt_C) # Reset fh_min_C
                fh_max_C = calculate_fh(k_C, L(lambda_C, t_c_max), l_tmelt_C) # Reset fh_max_C
                
                t_c_min, t_c_max, fh_min_B, fh_max_B, fh_min_C, fh_max_C = find_tc(t_c_min, t_c_max,
                                                                                   l_tmelt_B, l_tmelt_C,
                                                                                   lambda_B, lambda_C,
                                                                                   k_B, k_C,
                                                                                   fh_min_B, fh_max_B,
                                                                                   fh_min_C, fh_max_C)
                
                # No convergence in any combination case
                if min(abs(t_c - t_c_min), abs(t_c - t_c_max)) < 2*epsilon:
                    # If no BC convergence
                    # All approaches have failed in this case
                    f_h = 0.0
                    
                    if r_A_saturated and r_B_saturated and not r_C_saturated:
                        t_c = ti_C
                    elif r_A_saturated and not r_B_saturated:
                        t_c = ti_B
                    else:
                        t_c = ti_A                
                        
                else:
                    # If BC Convergence
                    fh_B = calculate_fh(k_B, L(lambda_B, t_c), l_tmelt_B)
                    fh_C = calculate_fh(k_C, L(lambda_C, t_c), l_tmelt_C)
                    f_h = (fh_B + fh_C) / 2.0                    
            else:
                # If AC Convergence
                fh_A = calculate_fh(k_A, L(lambda_A, t_c), l_tmelt_A)
                fh_C = calculate_fh(k_C, L(lambda_C, t_c), l_tmelt_C)
                f_h = (fh_A + fh_C) / 2.0        
        else:
            # If AB Convergence
            fh_A = calculate_fh(k_A, L(lambda_A, t_c), l_tmelt_A)
            fh_B = calculate_fh(k_B, L(lambda_C, t_c), l_tmelt_C)
            f_h = (fh_A + fh_B) / 2.0
                    
        # Calculate effective temperature
        if f_h > 0.0: # Solution found
            t_h = calculateTh(lambda_A, tau_A, t_c, rad_thermal_A, f_h)
            t_e = calculateTe(f_h, t_h, t_c)       
        else:
            t_e = t_c
            
        # Calculate Q
        q = calculateQ(A, e_q, sigma, t_e)
            
        # Create Row
        rowinfo = []
        rowinfo.append(rad_pixel_A)
        rowinfo.append(rad_pixel_B)
        rowinfo.append(rad_pixel_C)
        rowinfo.append(rad_thermal_A)
        rowinfo.append(rad_thermal_B)
        rowinfo.append(rad_thermal_C)
        rowinfo.append(ti_A)
        rowinfo.append(ti_B)
        rowinfo.append(ti_C)
        rowinfo.append('yes' if r_A_saturated else 'no')
        rowinfo.append('yes' if r_B_saturated else 'no')
        rowinfo.append('yes' if r_C_saturated else 'no')
        rowinfo.append('yes' if r_A_too_small else 'no')
        rowinfo.append('yes' if r_B_too_small else 'no')
        rowinfo.append('yes' if r_C_too_small else 'no')
        rowinfo.append(f_h)
        rowinfo.append(1.0 - f_h)
        rowinfo.append(t_c)
        rowinfo.append(t_e)
        rowinfo.append(q)
    
        insert_row(outfile, rowinfo)

In [9]:
def determinePixel(outfile, radA, radB, radC=[]):
    """Control sequence allowing for situations where there 
    are two or three values
    @param outfile name of csv to output
    @param radA array of values
    @param radB array of values
    @param radC array of values or None
    """
    global row_titles
    
    if len(radC) > 0:
        row_titles = ['RawA', 'RawB', 'RawC','R_A', 'R_B', 'R_C' 'T_i_A', 'T_i_B', 'T_i_C', 'R_A_sat', 'R_B_sat', 'R_C_sat',
                      'R_A_too_small', 'R_B_too_small', 'R_C_too_small', 'f_h', 'f_c', 'T_c', 'T_e', 'Q']
        pixelTriBand(radA, radB, radC)
    else:
        row_titles = ['Raw_5', 'Raw_7', 'R_5', 'R_7', 'T_i_5', 'T_i_7', 'R_5_sat', 'R_7_sat', 'R_5_too_small', 'R_7_too_small',
                      'f_h', 'f_c', 'T_c', 'T_e', 'Q']
        create_csv(outfile)
        pixelDualBand(radA, radB)

# Running

## Dual Band

In [14]:
import pandas as pd

data = pd.read_excel('data.xls')
print(data)

# Info is in column 5 and 6 named 'Unnamed: 5' and 'Unnamed: 6'
# Use arrays (Can impliment most of the above in array form which will be much faster)
rad_5 = data['Unnamed: 5'].dropna().to_numpy()
rad_7 = data['Unnamed: 6'].dropna().to_numpy()

# First 2 enteries are the titles so remove them
rad_5_array = rad_5[2:].flatten()
rad_7_array = rad_7[2:].flatten()

     Unnamed: 0  Unnamed: 1  Unnamed: 2  Unnamed: 3  Unnamed: 4 Unnamed: 5  \
0           NaN         NaN         NaN         NaN         NaN        NaN   
1           NaN         NaN         NaN         NaN         NaN        NaN   
2           NaN         NaN         NaN         NaN         NaN        NaN   
3           NaN         NaN         NaN         NaN         NaN        NaN   
4           NaN         NaN         NaN         NaN         NaN        NaN   
..          ...         ...         ...         ...         ...        ...   
544         NaN         NaN         NaN         NaN         NaN     2.8244   
545         NaN         NaN         NaN         NaN         NaN     2.6332   
546         NaN         NaN         NaN         NaN         NaN     2.0595   
547         NaN         NaN         NaN         NaN         NaN     4.9278   
548         NaN         NaN         NaN         NaN         NaN     4.9278   

    Unnamed: 6  
0          NaN  
1          NaN  
2          N

Use band 5 and 7 here so set them

In [15]:
lambda_A = 1.65; r_A_min = -7.67183; r_A_max = 92.90148; bg_A = 0.
lambda_B = 2.22; r_B_min = -2.58582; r_B_max = 31.31277; bg_B = 0.

# Not sure what to set these to
e_q = 1 
sigma = 1
A = 400

outfile = 'output2.csv'

determinePixel(outfile, rad_5_array, rad_7_array)

In [16]:
xl = pd.read_csv('output2.csv')
print(xl)

       Raw_5    Raw_7      R_5      R_7       T_i_5       T_i_7 R_5_sat  \
0     1.6771  10.8213   1.6771  10.8213  585.030797  560.787951      no   
1     0.5298   5.1692   0.5298   5.1692  543.047750  527.093241      no   
2     2.6332   7.8955   2.6332   7.8955  603.290506  545.898521      no   
3    10.0908  12.6832  10.0908  12.6832  665.108548  568.598811      no   
4    20.9903  16.5400  20.9903  16.5400  704.463858  582.158964      no   
..       ...      ...      ...      ...         ...         ...     ...   
533   2.8244   8.0950   2.8244   8.0950  606.230467  547.048316      no   
534   2.6332   8.2280   2.6332   8.2280  603.290506  547.801827      no   
535   2.0595   5.2357   2.0595   5.2357  593.205302  527.641766      no   
536   4.9278  13.0157   4.9278  13.0157  630.633195  569.892643      no   
537   4.9278  11.7523   4.9278  11.7523  630.633195  564.821463      no   

    R_7_sat R_5_too_small R_7_too_small       f_h       f_c         T_c  \
0        no            n

## TriBand

In [18]:
# This section wont work until a third rad is added
import pandas as pd

data = pd.read_excel('data3band.xls')
print(data)

# Info is in column 5 and 6 named 'Unnamed: 5' and 'Unnamed: 6'
# Use arrays (Can impliment most of the above in array form which will be much faster)
rad_5 = data['B5'].dropna().to_numpy()
rad_6 = data['B6'].dropna().to_numpy()
rad_7 = data['B7'].dropna().to_numpy()

# First 2 enteries are the titles so remove them
rad_5_array = rad_5[2:].flatten()
rad_6_array = rad_6[2:].flatten()
rad_7_array = rad_7[2:].flatten()

          B5       B6       B7
0    25.8132   8.8016  10.4282
1    13.7921  17.1888  23.5415
2    15.7915  34.5969  31.3130
3    17.7539   9.2989  14.1339
4    43.4561  62.1923  26.6192
..       ...      ...      ...
667  17.3651   5.0033   7.3846
668  16.9579   5.3071   7.4208
669  19.4509   5.7123   8.1217
670  18.1797   6.2525  10.1023
671  18.6364   5.7322   8.8158

[672 rows x 3 columns]


In [23]:
lambda_A = 1.65; r_A_min = -7.67183; r_A_max = 92.90148; bg_A = 0.
lambda_B = 2.22; r_B_min = -2.58582; r_B_max = 31.31277; bg_B = 0.
lambda_C = 1.65; r_C_min = -7.67183; r_C_max = 92.90148; bg_C = 0.


# Not sure what to set these to
e_q = 1 
sigma = 1
A = 400
tau_A = 0.90

outfile = 'output3.csv'

determinePixel(outfile, rad_5_array, rad_6_array, rad_7_array)

/global/homes/s/seriksen/miniconda3/envs/fast_lz/lib/python3.7/site-packages/ipykernel_launcher.py:117: RuntimeWarning: invalid value encountered in log


In [22]:
xl = pd.read_csv('output3.csv')
print(xl)

ParserError: Error tokenizing data. C error: Expected 15 fields in line 672, saw 20
